In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [398]:
### Scrape All ID

In [400]:
url = 'https://www.oktatas.hu/kozneveles/intezmenykereso/intezmenyek_a_szamok_tukreben/erettsegi_statisztikak/!DARI_ErettsegiStat/oh.php?id=kir_erett_talalat&oldal='
head = {'Cookie' : 'wcmid=user#21976512; DARI_ErettsegiStat-sessionid=uj5a81vqh5ll064p9tncv32tl2; Mainappstate-DARI_ErettsegiStat=%2Foh.php; DARI_Intezmenykereso-sessionid=c4vvvhg6g9vivaupgir6b0f695; Mainappstate-DARI_Intezmenykereso=%2Foh.php; Mainappstate-KIR_Intezmenykereso=%2FIntezmeny%2FIndex%2F035218'}
OMs = []
pages = 23
for i in range(pages):
    res = requests.get(url + str(i+1), headers=head)
    soup = BeautifulSoup(res.content)
    OMs.extend([tr.find_all('td')[1].text for tr in soup.find_all('tr')[2:-1]])
    print(f'The {i+1}. page Done!')

The 1. page Done!
The 2. page Done!
The 3. page Done!
The 4. page Done!
The 5. page Done!
The 6. page Done!
The 7. page Done!
The 8. page Done!
The 9. page Done!
The 10. page Done!
The 11. page Done!
The 12. page Done!
The 13. page Done!
The 14. page Done!
The 15. page Done!
The 16. page Done!
The 17. page Done!
The 18. page Done!
The 19. page Done!
The 20. page Done!
The 21. page Done!
The 22. page Done!
The 23. page Done!


In [406]:
df = pd.DataFrame(data=OMs)

In [469]:
df.to_csv('./SchoolData/omid_to_collect.csv', index=False)

In [450]:
cols = ['school_id',
                    'school_name ',
                    'school_dean' ,
                    'school_dean_title',
                    'school_address',
                    'school_type',
                    'no_hs_student',
                    'no_hs_student_female' ,
                    'no_rooms ',
                    'no_classes',
                    'no_fail', 
                    'fulltime_teachers',
                    'parttime_teachers',
                    'total_teachers',
                    'first_place_acceptance_rate', 
                    'acceptance_rate',
                    'hu_gcse_pop','hu_gcse_perc','hu_gcse_avg',
                    'hu_A_pop','hu_A_perc','hu_A_avg',
                    'math_gcse_pop','math_gcse_perc','math_gcse_avg',
                    'math_A_pop','math_A_perc','math_A_avg', 
                    'hist_gcse_pop','hist_gcse_perc','hist_gcse_avg',
                    'hist_A_pop','hist_A_perc','hist_A_avg',
                    'eng_gcse_pop','eng_gcse_perc','eng_gcse_avg',
                    'eng_A_pop','eng_A_perc','eng_A_avg',
                    'math_competency','lit_competency']

df_schools = pd.DataFrame(columns=cols)#

for om in  list(set(OMs)):  
    url = 'https://www.oktatas.hu/kozneveles/intezmenykereso/koznevelesi_intezmenykereso/!DARI_Intezmenykereso/oh.php?id=kir_int_mod&int_omkod=' + om
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    soup = BeautifulSoup(res.content)
    #Fotablazat
    try:
        df_admin = pd.read_html(str(soup.findAll('table')[2]), index_col=1)[0]

        ## Data gather from admin table: 

        school_id = df_admin.loc['Az intézmény OM azonosítója:'][2]
        school_name = df_admin.loc['Az intézmény megnevezése:'][2]
        school_dean = df_admin.loc['Az intézmény vezetője:'][2]
        school_dean_title = df_admin.loc['Beosztása:'].iloc[0][2]
        school_address = df_admin.loc['Címe:'][2]
        school_type = df_admin.loc['Ellátott feladatok:'][2].split(',')

        ##Basic Quantitative Data
        #### Basic Stats on Children
        t_basics = soup.find('div', id='adata01t01').find('table')
        t_teachers = soup.find('div', id='adata01t06').find('table')
        t_students = soup.find('div', id='adata04t18').find('table')

        ##Basics
        df_basics  = pd.read_html(str(t_basics),header=0, index_col=0)[0]

        no_hs_student = df_basics['Gyermekek, tanulók nyitó létszáma'].loc['Gimnázium']  
        no_hs_student_female = df_basics['Gyermekek, tanulók nyitó létszáma.1'].loc['Gimnázium']  
        no_rooms = df_basics['Osztályterem, szaktanterem / csoportszoba összesen'].loc['Gimnázium']
        no_classes= df_basics['Osztályok, csoportok száma összesen'].loc['Gimnázium']  
        df_basics  = pd.read_html(str(t_basics),header=0, index_col=0)[0]

        no_hs_student = df_basics['Gyermekek, tanulók nyitó létszáma'].loc['Gimnázium']  
        no_hs_student_female = df_basics['Gyermekek, tanulók nyitó létszáma.1'].loc['Gimnázium']  
        no_rooms = df_basics['Osztályterem, szaktanterem / csoportszoba összesen'].loc['Gimnázium']
        no_classes= df_basics['Osztályok, csoportok száma összesen'].loc['Gimnázium']  
        ###Students
        df_students = pd.read_html(str(t_students), index_col=[1,2])[0]

        ### tons of important info ignored here
        no_fail = df_students.loc[('évfolyamismétlő','évfolyamismétlő')][3]
        ###Teachers
        df_teachers = pd.read_html(str(t_teachers), index_col=0, header=0)[0]
        fulltime_teachers = df_teachers['Összesen'].loc['Teljes munkaidős']
        parttime_teachers = df_teachers['Összesen'].loc['Részmunkaidős']
        total_teachers = df_teachers['Összesen'].loc['Összesen']
        ##Competency test
        ### Competency Test - get the average of all recent results

        t_iq = soup.find('div', id='adat_okm2019').find('table')
        df_competency = pd.read_html(str(t_iq),index_col=3, header=1)[0]

        math_competency = df_competency['Matematika'].mean()
        lit_competency = df_competency['Szövegértés'].mean()
        ###Erettsegi test
        ### Erettsegi Tables - GSCI

        t_2018= soup.find('div', id='adat_stat2018_vtargy').find('table')
        t_2019= soup.find('div', id='adat_stat2019_vtargy').find('table')
        t_2020= soup.find('div', id='adat_stat2020_vtargy').find('table')
        df_erett_2020 = pd.read_html(str(t_2020), header=0)[0]
        hu_gcse_pop,hu_gcse_perc,hu_gcse_avg 
        hu_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='magyar nyelv és irodalom') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        hu_gcse_pop,hu_gcse_perc,hu_gcse_avg = hu_gcse[0] if len(hu_gcse)==1 else [None,None,None]

        hu_A = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='magyar nyelv és irodalom') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        hu_A_pop,hu_A_perc,hu_A_avg = hu_A[0] if len(hu_A)==1 else [None,None,None]

        math_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='matematika') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        math_gcse_pop,math_gcse_perc,math_gcse_avg = math_gcse[0] if len(math_gcse)==1 else [None,None,None]

        math_A = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='matematika') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        math_A_pop,math_A_perc,math_A_avg = math_A[0] if len(math_A)==1 else [None,None,None]

        hist_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='történelem') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        hist_gcse_pop,hist_gcse_perc,hist_gcse_avg = hist_gcse[0] if len(hist_gcse)==1 else [None,None,None]

        hist_A = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='történelem') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        hist_A_pop,hist_A_perc,hist_A_avg = hist_A[0] if len(hist_A)==1 else [None,None,None]

        eng_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='angol nyelv') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        eng_gcse_pop,eng_gcse_perc,eng_gcse_avg = eng_gcse[0] if len(eng_gcse)==1 else [None,None,None]

        eng_A= df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='angol nyelv') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
        eng_A_pop,eng_A_perc,eng_A_avg = eng_A[0] if len(eng_A)==1 else [None,None,None]

        ##Acceptence Rate
        ### Enrollment data
        t_enroll2019 = soup.find('div', id='adat_kifir2019').find('table')
        first_place_acceptance_rate
        ## Calcualte applied and accepted rates

        enroll2019 = pd.read_html(str(t_enroll2019), header=0)[0]

        first_place_acceptance_rate = enroll2019['1. helyen jelentkezettek közül a felvettek száma'].sum()/ enroll2019['Jelentkezők száma'].sum()
        acceptance_rate = enroll2019['Felvettek száma'].sum() / enroll2019['Jelentkezők száma'].sum()

        df_school = pd.DataFrame(data=[school_id,
                                    school_name ,
                                    school_dean ,
                                    school_dean_title,
                                    school_address,
                                    school_type,
                                    no_hs_student,
                                    no_hs_student_female ,
                                    no_rooms ,
                                    no_classes,
                                    no_fail, 
                                    fulltime_teachers,
                                    parttime_teachers,
                                    total_teachers,
                                    first_place_acceptance_rate, 
                                    acceptance_rate,
                                    hu_gcse_pop,hu_gcse_perc,hu_gcse_avg,
                                    hu_A_pop,hu_A_perc,hu_A_avg,
                                    math_gcse_pop,math_gcse_perc,math_gcse_avg,
                                    math_A_pop,math_A_perc,math_A_avg, 
                                    hist_gcse_pop,hist_gcse_perc,hist_gcse_avg,
                                    hist_A_pop,hist_A_perc,hist_A_avg,
                                    eng_gcse_pop,eng_gcse_perc,eng_gcse_avg,
                                    eng_A_pop,eng_A_perc,eng_A_avg,
                                    math_competency,lit_competency
                                    ],index=cols).T

        df_schools=pd.concat([df_schools,df_school])
        print(om + ' - Done!')
    except:
        print(om + ' - FAIL!')
        pass
   
    

200598 - Done!
035357 - Done!
201521 - FAIL!
200518 - Done!
200672 - FAIL!
101503 - Done!
035452 - FAIL!
203078 - FAIL!
200406 - Done!
036416 - Done!
035249 - Done!
100534 - Done!
036757 - FAIL!
034923 - Done!
203175 - Done!
027264 - Done!
028456 - Done!
031936 - Done!
031986 - FAIL!
035260 - Done!
202736 - FAIL!
032555 - Done!
035278 - Done!
201400 - Done!
036394 - Done!
032552 - Done!
201600 - FAIL!
036005 - FAIL!
035244 - Done!
035377 - FAIL!
037775 - Done!
200078 - Done!
035237 - Done!
030180 - Done!
200328 - Done!
028280 - Done!
100523 - Done!
037178 - Done!
035242 - Done!
203039 - FAIL!
030696 - Done!
037172 - Done!
201799 - Done!
037640 - Done!
030695 - Done!
031597 - Done!
035323 - Done!
035384 - FAIL!
200290 - Done!
031937 - Done!
031197 - Done!
101641 - Done!
201640 - FAIL!
203047 - FAIL!
200868 - Done!
036396 - FAIL!
031198 - Done!
201260 - Done!
027425 - Done!
031619 - FAIL!
202723 - FAIL!
203038 - FAIL!
201173 - Done!
032655 - FAIL!
034146 - Done!
035491 - FAIL!
200415 - D

<ipython-input-450-cdbd81e95ac8>:123: RuntimeWarning: invalid value encountered in longlong_scalars
  first_place_acceptance_rate = enroll2019['1. helyen jelentkezettek közül a felvettek száma'].sum()/ enroll2019['Jelentkezők száma'].sum()
<ipython-input-450-cdbd81e95ac8>:124: RuntimeWarning: invalid value encountered in longlong_scalars
  acceptance_rate = enroll2019['Felvettek száma'].sum() / enroll2019['Jelentkezők száma'].sum()


035890 - Done!
035301 - FAIL!
201690 - FAIL!
203195 - Done!
203378 - FAIL!
200963 - Done!
028375 - Done!
202740 - FAIL!
203044 - FAIL!
035256 - Done!
035187 - Done!
032450 - Done!
033645 - Done!
035327 - Done!
036728 - Done!
032564 - Done!
201188 - Done!
030491 - FAIL!
036410 - FAIL!
031631 - Done!
032569 - Done!
201505 - Done!
029741 - FAIL!
032575 - Done!
203117 - Done!
035239 - Done!
102290 - Done!
032579 - Done!
200329 - Done!
035279 - Done!
200483 - FAIL!


<ipython-input-450-cdbd81e95ac8>:123: RuntimeWarning: invalid value encountered in longlong_scalars
  first_place_acceptance_rate = enroll2019['1. helyen jelentkezettek közül a felvettek száma'].sum()/ enroll2019['Jelentkezők száma'].sum()
<ipython-input-450-cdbd81e95ac8>:124: RuntimeWarning: invalid value encountered in longlong_scalars
  acceptance_rate = enroll2019['Felvettek száma'].sum() / enroll2019['Jelentkezők száma'].sum()


035273 - Done!
200031 - FAIL!
036378 - Done!
201189 - Done!
027942 - Done!
031960 - Done!
031203 - Done!
027939 - Done!
202755 - Done!
200893 - Done!
203265 - FAIL!
200667 - Done!
035220 - Done!
037170 - Done!
203061 - FAIL!
029340 - FAIL!
027946 - Done!
035305 - Done!
200627 - Done!
033646 - Done!
203196 - FAIL!
035319 - Done!
201676 - Done!
032295 - Done!
029265 - Done!
032600 - Done!
035348 - Done!
064887 - Done!
035843 - Done!
039554 - FAIL!
027943 - Done!
062576 - FAIL!
034155 - Done!
036725 - Done!
034853 - Done!
200923 - Done!
201038 - FAIL!
202801 - Done!
200708 - Done!
201594 - FAIL!
202722 - Done!
036723 - Done!
039227 - Done!
035322 - Done!
202724 - FAIL!
032464 - Done!
032568 - Done!
034145 - Done!
033661 - Done!
100565 - Done!
037206 - Done!
203204 - FAIL!
203030 - FAIL!
037630 - Done!
036360 - Done!
203069 - FAIL!
200838 - Done!
100537 - Done!
201474 - FAIL!
203048 - FAIL!
028382 - Done!
203109 - FAIL!
030546 - Done!
203027 - FAIL!
035990 - Done!
035994 - Done!


KeyboardInterrupt: 

In [451]:
 list(set(OMs)).index(om)

234

In [460]:
166/234
df_schools.to_json('./SchoolData/schools_test.json', orient='records')

In [461]:
df_schools.columns

Index(['school_id', 'school_name ', 'school_dean', 'school_dean_title',
       'school_address', 'school_type', 'no_hs_student',
       'no_hs_student_female', 'no_rooms ', 'no_classes', 'no_fail',
       'fulltime_teachers', 'parttime_teachers', 'total_teachers',
       'first_place_acceptance_rate', 'acceptance_rate', 'hu_gcse_pop',
       'hu_gcse_perc', 'hu_gcse_avg', 'hu_A_pop', 'hu_A_perc', 'hu_A_avg',
       'math_gcse_pop', 'math_gcse_perc', 'math_gcse_avg', 'math_A_pop',
       'math_A_perc', 'math_A_avg', 'hist_gcse_pop', 'hist_gcse_perc',
       'hist_gcse_avg', 'hist_A_pop', 'hist_A_perc', 'hist_A_avg',
       'eng_gcse_pop', 'eng_gcse_perc', 'eng_gcse_avg', 'eng_A_pop',
       'eng_A_perc', 'eng_A_avg'],
      dtype='object')

In [473]:
df_schools.head(10)

,school_id,school_name,school_dean,school_dean_title,school_address,school_type,no_hs_student,no_hs_student_female,no_rooms,no_classes,...,hist_gcse_avg,hist_A_pop,hist_A_perc,hist_A_avg,eng_gcse_pop,eng_gcse_perc,eng_gcse_avg,eng_A_pop,eng_A_perc,eng_A_avg
0,200598,Encsi Váci Mihály Gimnázium és Kollégium,Karsai Attila,intézményvezető,"3860 Encs, Petőfi Sándor út 60.","[4 évfolyamos gimnáziumi nevelés-oktatás, 6 é...",329,181,20,13,...,3.32,6,68 %,4.83,31,59 %,3.35,5,46 %,3.6
0,035357,Budapesti Egyetemi Katolikus Gimnázium és Koll...,Endrédi Józsefné,intézményvezető,"1015 Budapest I. kerület, Szabó Ilonka utca 2-4.","[4 évfolyamos gimnáziumi nevelés-oktatás, 6 é...",731,360,31,23,...,3.68,25,67 %,4.6,58,66 %,3.67,40,65 %,4.53
0,200518,Budapesti Montessori Általános Iskola és Gimná...,Juhász Edit,intézményvezető,"1203 Budapest XX. kerület, Kálmán utca 15.",[általános iskolai nevelés-oktatás (alsó tagoz...,73,35,7,4,...,2.1,5-nél kevesebb,47 %,3.5,5,62 %,3.8,5-nél kevesebb,60 %,4.33
0,101503,"Forrás Waldorf Általános Iskola, Gimnázium és ...",Márkus Andrea Éva,igazgató,"9027 Győr, Madách utca 10.",[általános iskolai nevelés-oktatás (alsó tagoz...,95,43,8,5,...,3.73,None,None,None,5-nél kevesebb,48 %,3,5-nél kevesebb,78 %,5
0,200406,"Koch Valéria Gimnázium, Általános Iskola, Óvod...",Amreinné Pesti Ágnes,intézményvezető,"7624 Pécs, Tiborc utca 28/1.","[óvodai nevelés, általános iskolai nevelés-ok...",280,165,15,10,...,3.71,5-nél kevesebb,65 %,5,9,86 %,4.78,12,73 %,4.58
0,036416,"Szekszárdi Kolping Katolikus Szakképző Iskola,...",Koller Tamás,intézményvezető,"7100 Szekszárd, Pázmány tér 4.","[szakképző iskola (kizárólag szakmai képzés), ...",55,28,3,2,...,2.62,None,None,None,5,33 %,2,None,None,None
0,035249,Budapest XVI. Kerületi Szerb Antal Gimnázium,Inotai István Károly,intézményvezető,"1164 Budapest XVI. kerület, Batthyány Ilona ut...","[4 évfolyamos gimnáziumi nevelés-oktatás, 6 é...",665,362,38,21,...,4.32,24,72 %,4.75,20,90 %,4.75,15,78 %,5
0,100534,Innovációs Gimnázium,Lénárt Péter József,intézményvezető,"1153 Budapest XV. kerület, Bocskai utca 70-72.",[4 évfolyamos gimnáziumi nevelés-oktatás],305,168,15,11,...,2.35,5-nél kevesebb,46 %,3,21,37 %,2.24,18,56 %,4
0,034923,Lakatos Menyhért Általános Iskola és Gimnázium,Csere Margit,mb. intézményvezető,"1086 Budapest VIII. kerület, Bauer Sándor utca...",[általános iskolai nevelés-oktatás (alsó tagoz...,70,33,10,7,...,2.67,None,None,None,12,35 %,2,None,None,None
0,203175,Szent Márton Katolikus Gimnázium és Általános ...,Nyitrainé Jurácsik Antónia,intézményvezető,"3800 Szikszó, Miskolci út 9.",[általános iskolai nevelés-oktatás (alsó tagoz...,270,135,13,11,...,3.81,6,71 %,4.83,16,58 %,3.31,5-nél kevesebb,79 %,5
